# Chapter 2: Type Annotations - Deep Dive

Building on Chapter 1's introduction, this notebook explores advanced type annotation concepts: TypeVar for generics, Protocol for structural typing, and type narrowing with isinstance checks.

## Section 1: TypeVar - Generic Types

In [ ]:
from typing import TypeVar

# TypeVar creates a type variable - a placeholder for any type
T = TypeVar('T')  # T can be any type

def get_first(items: list[T]) -> T:
    """Get the first item from a list.
    
    The return type is the same as the list's element type.
    If you pass list[int], it returns int.
    If you pass list[str], it returns str.
    """
    return items[0]

# mypy knows the return type matches the input type
first_number = get_first([1, 2, 3])  # Inferred as int
first_word = get_first(["a", "b", "c"])  # Inferred as str

print(f"first_number: {first_number}, type: {type(first_number).__name__}")
print(f"first_word: {first_word}, type: {type(first_word).__name__}")

In [ ]:
# Constrained TypeVar - limit which types are allowed
Numeric = TypeVar('Numeric', int, float)  # Only int or float

def multiply_by_two(value: Numeric) -> Numeric:
    """Multiply a number by two.
    
    TypeVar preserves the input type:
    - int input returns int
    - float input returns float
    """
    return value * 2  # type: ignore

result_int = multiply_by_two(5)      # Returns int
result_float = multiply_by_two(5.5)  # Returns float

print(f"multiply_by_two(5) = {result_int} (type: {type(result_int).__name__})")
print(f"multiply_by_two(5.5) = {result_float} (type: {type(result_float).__name__})")

# This would be a type error (mypy would catch it):
# multiply_by_two("hello")  # Error: str not allowed

In [ ]:
# Generic classes using TypeVar
class Container(list[T]):
    """A container that preserves element type."""
    
    def first(self) -> T:
        if not self:
            raise IndexError("Container is empty")
        return self[0]
    
    def last(self) -> T:
        if not self:
            raise IndexError("Container is empty")
        return self[-1]

# Create a container of integers
int_container = Container([1, 2, 3, 4, 5])
print(f"First: {int_container.first()}, Last: {int_container.last()}")

# Create a container of strings
str_container = Container(["apple", "banana", "cherry"])
print(f"First: {str_container.first()}, Last: {str_container.last()}")

## Section 2: Protocol - Structural Typing

In [ ]:
from typing import Protocol

# Protocol defines a structural interface (duck typing with type checking)
class Drawable(Protocol):
    """Anything that has a draw() method."""
    
    def draw(self) -> None:
        ...

# These classes don't explicitly inherit from Drawable
# but they satisfy the protocol by having a draw() method

class Circle:
    def __init__(self, radius: float) -> None:
        self.radius = radius
    
    def draw(self) -> None:
        print(f"Drawing circle with radius {self.radius}")

class Square:
    def __init__(self, side: float) -> None:
        self.side = side
    
    def draw(self) -> None:
        print(f"Drawing square with side {self.side}")

# Function accepts anything that implements Drawable protocol
def render(shape: Drawable) -> None:
    """Render any drawable shape."""
    shape.draw()

# Both Circle and Square work, even though they don't inherit Drawable
circle = Circle(5)
square = Square(4)

render(circle)
render(square)

In [ ]:
# Protocol for objects that can be converted to JSON-like format
class JSONSerializable(Protocol):
    """Anything that can be serialized to JSON."""
    
    def to_dict(self) -> dict:
        ...

class User:
    def __init__(self, name: str, email: str) -> None:
        self.name = name
        self.email = email
    
    def to_dict(self) -> dict:
        return {"name": self.name, "email": self.email}

class Product:
    def __init__(self, name: str, price: float) -> None:
        self.name = name
        self.price = price
    
    def to_dict(self) -> dict:
        return {"name": self.name, "price": self.price}

def serialize(obj: JSONSerializable) -> str:
    """Convert any JSONSerializable to a string."""
    import json
    return json.dumps(obj.to_dict())

user = User("Alice", "alice@example.com")
product = Product("Laptop", 999.99)

print(f"User: {serialize(user)}")
print(f"Product: {serialize(product)}")

## Section 3: Overload - Multiple Signatures

In [ ]:
from typing import overload

# overload allows defining multiple type signatures for one function
# The actual implementation comes after all overloads

@overload
def process(value: int) -> int:
    ...

@overload
def process(value: str) -> str:
    ...

@overload
def process(value: list) -> int:
    ...

# Actual implementation (doesn't have @overload)
def process(value):
    """Process different types differently."""
    if isinstance(value, int):
        return value * 2
    elif isinstance(value, str):
        return value.upper()
    elif isinstance(value, list):
        return len(value)
    return None

# mypy knows the return types
int_result = process(5)           # mypy knows this is int
str_result = process("hello")     # mypy knows this is str
len_result = process([1, 2, 3])   # mypy knows this is int

print(f"process(5) = {int_result} (type: {type(int_result).__name__})")
print(f"process('hello') = {str_result} (type: {type(str_result).__name__})")
print(f"process([1,2,3]) = {len_result} (type: {type(len_result).__name__})")

## Section 4: Type Guards and Narrowing

In [ ]:
# Type narrowing with isinstance and control flow

def process_value(value: int | str | list) -> str:
    """Process a value that could be int, str, or list."""
    
    # isinstance narrows the type
    if isinstance(value, int):
        # Inside this block, mypy knows value is int
        return f"Integer: {value * 2}"
    elif isinstance(value, str):
        # Inside this block, mypy knows value is str
        return f"String: {value.upper()}"
    else:
        # Remaining possibility is list
        return f"List of {len(value)} items"

print(process_value(42))
print(process_value("hello"))
print(process_value([1, 2, 3]))

In [ ]:
from typing import TypeGuard

# TypeGuard: custom type narrowing function
def is_non_empty_string(value: str | None) -> TypeGuard[str]:
    """Check if value is a non-empty string.
    
    TypeGuard tells mypy that if this returns True,
    then value is definitely a str (not None).
    """
    return isinstance(value, str) and len(value) > 0

def greet(name: str | None) -> str:
    # First check
    if is_non_empty_string(name):
        # mypy now knows name is str, not None
        return f"Hello, {name.upper()}!"
    else:
        return "Hello, stranger!"

print(greet("Alice"))
print(greet(None))
print(greet(""))

## Section 5: Generic Classes

In [ ]:
from typing import Generic, TypeVar

# Define a type variable
T = TypeVar('T')

# Create a generic class
class Stack(Generic[T]):
    """A type-safe stack implementation."""
    
    def __init__(self) -> None:
        self._items: list[T] = []
    
    def push(self, item: T) -> None:
        """Add item to stack."""
        self._items.append(item)
    
    def pop(self) -> T:
        """Remove and return the top item."""
        if not self._items:
            raise IndexError("Stack is empty")
        return self._items.pop()
    
    def is_empty(self) -> bool:
        """Check if stack is empty."""
        return len(self._items) == 0
    
    def size(self) -> int:
        """Get stack size."""
        return len(self._items)

# Stack of integers
int_stack: Stack[int] = Stack()
int_stack.push(1)
int_stack.push(2)
int_stack.push(3)

print(f"Popped: {int_stack.pop()}")
print(f"Size: {int_stack.size()}")

# Stack of strings
str_stack: Stack[str] = Stack()
str_stack.push("first")
str_stack.push("second")

print(f"\nPopped: {str_stack.pop()}")
print(f"Size: {str_stack.size()}")

In [ ]:
# Generic class with multiple type variables
K = TypeVar('K')  # Key
V = TypeVar('V')  # Value

class Pair(Generic[K, V]):
    """A key-value pair with type safety."""
    
    def __init__(self, key: K, value: V) -> None:
        self.key = key
        self.value = value
    
    def swap(self) -> 'Pair[V, K]':
        """Return a pair with key and value swapped."""
        return Pair(self.value, self.key)
    
    def __repr__(self) -> str:
        return f"Pair({self.key!r}, {self.value!r})"

# Pair of int and str
pair1 = Pair(1, "one")
print(f"Original: {pair1}")
print(f"Swapped: {pair1.swap()}")

# Pair of str and float
pair2 = Pair("pi", 3.14159)
print(f"\nOriginal: {pair2}")
print(f"Swapped: {pair2.swap()}")

## Section 6: Callable with Constraints

In [ ]:
from typing import Callable

# Callable type for callbacks
def process_data(
    data: list[int],
    processor: Callable[[int], int]
) -> list[int]:
    """Apply a processor function to each element."""
    return [processor(item) for item in data]

# Different callbacks
def double(x: int) -> int:
    return x * 2

def square(x: int) -> int:
    return x ** 2

data = [1, 2, 3, 4, 5]

print(f"Original: {data}")
print(f"Doubled: {process_data(data, double)}")
print(f"Squared: {process_data(data, square)}")
print(f"Lambda: {process_data(data, lambda x: x + 10)}")

## Summary

### Key Concepts
1. **TypeVar**: Generic type variables preserve input/output relationships
   - Unconstrained: `T = TypeVar('T')`
   - Constrained: `Numeric = TypeVar('Numeric', int, float)`

2. **Protocol**: Structural typing - any class with required methods matches
   - Enables duck typing with static type checking
   - No explicit inheritance needed

3. **@overload**: Multiple type signatures for one function
   - Provides better IDE support and type checking
   - Implementation comes after all overloads

4. **Type Narrowing**: Control flow narrows types
   - `isinstance()`, comparison, attribute checks
   - `TypeGuard` for custom type narrowing

5. **Generic Classes**: Parameterized with type variables
   - `class MyClass(Generic[T]):`
   - Multiple type variables: `Generic[K, V]`

### Best Practices
- Use `TypeVar` for functions that work with any type
- Use `Protocol` to define interface requirements
- Use `@overload` when behavior differs by type
- Leverage type narrowing to reduce `isinstance()` checks
- Use `Generic` for data structures and containers